In [1]:
import finnhub
import pandas as pd
import os
from dotenv import load_dotenv
import json
import datetime
from datetime import datetime, timedelta
import time

load_dotenv()
finnhub_api_key = os.getenv("FINNHUB_API_KEY")
if type(finnhub_api_key) == str:
    print('API OK')
else:
    print('API NOT OK', type(finnhub_api_key))
    print('Check your .env file for the FINNHUB_API_KEY value.')
    print('Sign-up and get an API key at https://finnhub.io/')

# Setup client
finnhub_client = finnhub.Client(api_key=finnhub_api_key)

API OK


In [2]:
symbol = pd.DataFrame(
    finnhub_client.crypto_symbols("COINBASE")
)
symbol

,description,displaySymbol,symbol
0,COINBASE MATIC/USD,MATIC/USD,COINBASE:MATIC-USD
1,COINBASE BCH/USD,BCH/USD,COINBASE:BCH-USD
2,COINBASE MKR/BTC,MKR/BTC,COINBASE:MKR-BTC
3,COINBASE SKL/GBP,SKL/GBP,COINBASE:SKL-GBP
4,COINBASE SKL/BTC,SKL/BTC,COINBASE:SKL-BTC
...,...,...,...
326,COINBASE MASK/USDT,MASK/USDT,COINBASE:MASK-USDT
327,COINBASE FARM/USD,FARM/USD,COINBASE:FARM-USD
328,COINBASE FX/USD,FX/USD,COINBASE:FX-USD
329,COINBASE RAD/EUR,RAD/EUR,COINBASE:RAD-EUR


In [3]:
# Time period of 86400 is going back 24 hours in seconds
beg_period = int(time.time()) - 86400
end_period = int(time.time())

btc_usd = pd.DataFrame(
    finnhub_client.crypto_candles("COINBASE:BTC-USD", "1", beg_period, end_period)
    )
btc_usd

,c,h,l,o,s,t,v
0,64096.22,64132.39,64096.21,64132.39,ok,1636922220,2.208560
1,64099.94,64099.94,64062.32,64096.22,ok,1636922280,1.684460
2,64095.17,64108.68,64090.49,64099.93,ok,1636922340,1.327987
3,64138.20,64138.20,64091.10,64091.10,ok,1636922400,0.934748
4,64100.80,64155.87,64100.80,64144.61,ok,1636922460,1.856071
...,...,...,...,...,...,...,...
1441,63838.16,63853.29,63833.29,63853.28,ok,1637008920,4.041341
1442,63838.98,63843.46,63801.28,63838.16,ok,1637008980,4.776516
1443,63833.05,63837.01,63807.36,63837.01,ok,1637009040,3.579583
1444,63910.88,64016.88,63802.02,63832.21,ok,1637009100,28.373572


In [4]:
import sqlalchemy as sql
from sqlalchemy import inspect
database_connection_string = 'sqlite:///'
engine = sql.create_engine(database_connection_string)
inspector = inspect(engine)
inspector.get_table_names()

[]

In [5]:
btc_usd.to_sql('btc_usd', engine, index=False, if_exists='replace')
inspector = inspect(engine)
inspector.get_table_names()

['btc_usd']

In [6]:
btc_usd_df = pd.read_sql_table('btc_usd', engine)
btc_usd_df.head()

,c,h,l,o,s,t,v
0,64096.22,64132.39,64096.21,64132.39,ok,1636922220,2.208560
1,64099.94,64099.94,64062.32,64096.22,ok,1636922280,1.684460
2,64095.17,64108.68,64090.49,64099.93,ok,1636922340,1.327987
3,64138.20,64138.20,64091.10,64091.10,ok,1636922400,0.934748
4,64100.80,64155.87,64100.80,64144.61,ok,1636922460,1.856071


In [7]:
btc_usd_df['t'] = pd.to_datetime(btc_usd_df['t'], unit='s')
btc_usd_df

,c,h,l,o,s,t,v
0,64096.22,64132.39,64096.21,64132.39,ok,2021-11-14 20:37:00,2.208560
1,64099.94,64099.94,64062.32,64096.22,ok,2021-11-14 20:38:00,1.684460
2,64095.17,64108.68,64090.49,64099.93,ok,2021-11-14 20:39:00,1.327987
3,64138.20,64138.20,64091.10,64091.10,ok,2021-11-14 20:40:00,0.934748
4,64100.80,64155.87,64100.80,64144.61,ok,2021-11-14 20:41:00,1.856071
...,...,...,...,...,...,...,...
1441,63838.16,63853.29,63833.29,63853.28,ok,2021-11-15 20:42:00,4.041341
1442,63838.98,63843.46,63801.28,63838.16,ok,2021-11-15 20:43:00,4.776516
1443,63833.05,63837.01,63807.36,63837.01,ok,2021-11-15 20:44:00,3.579583
1444,63910.88,64016.88,63802.02,63832.21,ok,2021-11-15 20:45:00,28.373572


In [8]:
btc_usd_df = btc_usd_df.set_index('t')
btc_usd_df

,c,h,l,o,s,v
t,,,,,,
2021-11-14 20:37:00,64096.22,64132.39,64096.21,64132.39,ok,2.208560
2021-11-14 20:38:00,64099.94,64099.94,64062.32,64096.22,ok,1.684460
2021-11-14 20:39:00,64095.17,64108.68,64090.49,64099.93,ok,1.327987
2021-11-14 20:40:00,64138.20,64138.20,64091.10,64091.10,ok,0.934748
2021-11-14 20:41:00,64100.80,64155.87,64100.80,64144.61,ok,1.856071
...,...,...,...,...,...,...
2021-11-15 20:42:00,63838.16,63853.29,63833.29,63853.28,ok,4.041341
2021-11-15 20:43:00,63838.98,63843.46,63801.28,63838.16,ok,4.776516
2021-11-15 20:44:00,63833.05,63837.01,63807.36,63837.01,ok,3.579583


In [9]:
# Create a DataFrame with the index and Close column from the dataset
signals_df = btc_usd_df.loc[:, ["c"]].copy()

# Set the short window and long windows
short_window = 1
long_window = 60

# Generate the short and long moving averages (50 and 200 days, respectively)
signals_df["Short_SMA"] = signals_df["c"].rolling(window=short_window).mean()
signals_df["Long_SMA"] = signals_df["c"].rolling(window=long_window).mean()

# Review the DataFrame
signals_df

,c,Short_SMA,Long_SMA
t,,,
2021-11-14 20:37:00,64096.22,64096.22,NaN
2021-11-14 20:38:00,64099.94,64099.94,NaN
2021-11-14 20:39:00,64095.17,64095.17,NaN
2021-11-14 20:40:00,64138.20,64138.20,NaN
2021-11-14 20:41:00,64100.80,64100.80,NaN
...,...,...,...
2021-11-15 20:42:00,63838.16,63838.16,64027.677167
2021-11-15 20:43:00,63838.98,63838.98,64021.084333
2021-11-15 20:44:00,63833.05,63833.05,64014.198500


In [10]:
import numpy as np

# Set the Signal column
signals_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the Short_SMA is greater than (or crosses over) the Long_SMA
# where 0 is when the Short_SMA is under the Long_SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df["Short_SMA"][short_window:] > signals_df["Long_SMA"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df

,c,Short_SMA,Long_SMA,Signal,Entry/Exit
t,,,,,
2021-11-14 20:37:00,64096.22,64096.22,NaN,0.0,NaN
2021-11-14 20:38:00,64099.94,64099.94,NaN,0.0,0.0
2021-11-14 20:39:00,64095.17,64095.17,NaN,0.0,0.0
2021-11-14 20:40:00,64138.20,64138.20,NaN,0.0,0.0
2021-11-14 20:41:00,64100.80,64100.80,NaN,0.0,0.0
...,...,...,...,...,...
2021-11-15 20:42:00,63838.16,63838.16,64027.677167,0.0,0.0
2021-11-15 20:43:00,63838.98,63838.98,64021.084333,0.0,0.0
2021-11-15 20:44:00,63833.05,63833.05,64014.198500,0.0,0.0


In [11]:
signals_df['Returns'] = signals_df['Entry/Exit'] * signals_df['c']
signals_df

,c,Short_SMA,Long_SMA,Signal,Entry/Exit,Returns
t,,,,,,
2021-11-14 20:37:00,64096.22,64096.22,NaN,0.0,NaN,NaN
2021-11-14 20:38:00,64099.94,64099.94,NaN,0.0,0.0,0.00
2021-11-14 20:39:00,64095.17,64095.17,NaN,0.0,0.0,0.00
2021-11-14 20:40:00,64138.20,64138.20,NaN,0.0,0.0,0.00
2021-11-14 20:41:00,64100.80,64100.80,NaN,0.0,0.0,0.00
...,...,...,...,...,...,...
2021-11-15 20:42:00,63838.16,63838.16,64027.677167,0.0,0.0,0.00
2021-11-15 20:43:00,63838.98,63838.98,64021.084333,0.0,0.0,0.00
2021-11-15 20:44:00,63833.05,63833.05,64014.198500,0.0,0.0,0.00


In [12]:
signals_df['Returns'].sum()

63052.22999999993

In [13]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["c"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["c"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["c"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["Short_SMA", "Long_SMA"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot

AttributeError: 'Series' object has no attribute 'hvplot'

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
df = btc_usd_df